In [ ]:
python -m spacy download en_core_web_sm

In [7]:
import os
import re
import parfit.parfit as pf

##Spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import spacy 
import en_core_web_sm
import string
from spacy.lang.en.stop_words import STOP_WORDS

import pandas as pd
import numpy as np
from numpy import asarray
from numpy import zeros
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

## preprocessing tools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold,GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,ParameterGrid
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,roc_auc_score, f1_score,make_scorer
from skopt import BayesSearchCV

## algorithm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

## tensorflow
# import tensorflow as tf
# from tensorflow.keras.datasets import imdb
# from keras.preprocessing.text import one_hot, Tokenizer
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Embedding
# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow import keras
# from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
# from keras.layers import Conv1D

#autenticating to google
# auth.authenticate_user()
# creds, _ = default()
# gc = gspread.authorize(creds)

nlp=spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')
nlp.pipe_names


df = pd.read_csv('stock_data.csv')
df = df.iloc[:,1:4]
df['Text'] = df['Text'].astype(str)
df['Date'] = pd.to_datetime(df['Date'])
df['Sentiment'] = df['Sentiment'].astype(np.int64)
print(df.info())

test_data = pd.read_csv('2020.csv')
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data= test_data.iloc[:,1:4]
test_data.rename(columns = {'Headline':'Text','Target':'Sentiment'}, inplace = True)
test_data['Text'] = test_data['Text'].astype(str)
test_data =test_data.replace(r'^\s*$', np.nan, regex=True)
test_data = test_data[['Text','Date']].merge(df, on=['Text','Date'], how='left')
test_data = test_data[test_data['Text'].str.contains("\?")==False]
test_data = test_data.loc[test_data['Sentiment'].isnull() == True][:5000]
print(test_data.info())
print(df['Sentiment'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26429 entries, 0 to 26428
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       26429 non-null  object        
 1   Date       26429 non-null  datetime64[ns]
 2   Sentiment  26429 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 619.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1435 entries, 6023 to 7492
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       1435 non-null   object        
 1   Date       1435 non-null   datetime64[ns]
 2   Sentiment  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 44.8+ KB
None
 1    16121
 0     6119
-1     4189
Name: Sentiment, dtype: int64


In [15]:
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [17]:
df.head()

,Text,Date,Sentiment,Tok_text
0,Sebi to control unsolicited fin market advise from social media influencers,2023-02-24,1,sebi control unsolicited fin market advise social media influencers
1,"Sebi pitches change in rules for REITs, InvITs; sponsors to own some units",2023-02-24,1,sebi pitches change rules reits invits sponsors units
2,Sri Lanka's body approves renewable energy projects of Adani group,2023-02-24,1,sri lanka body approves renewable energy projects adani group
3,Sebi likely to scrap small town-linked incentive for mutual funds,2023-02-24,1,sebi likely scrap small town linked incentive mutual funds
4,Indices post biggest weekly decline since June; Sensex falls 2.5%,2023-02-24,-1,indices post biggest weekly decline june sensex falls


In [21]:
data1 = df['Text'].str.replace('[^\w\s]', ' ')
data1 = " "+data1+" "
data1.info()

<class 'pandas.core.series.Series'>
RangeIndex: 26429 entries, 0 to 26428
Series name: Text
Non-Null Count  Dtype 
--------------  ----- 
26429 non-null  object
dtypes: object(1)
memory usage: 206.6+ KB


C:\Users\kings\AppData\Local\Temp\ipykernel_12988\2740825243.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data1 = df['Text'].str.replace('[^\w\s]', ' ')


In [35]:
data = pd.read_csv('company.csv')
data['Company Name'] = data['Company Name'].apply(str).apply(str.lower)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2113 entries, 0 to 2112
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Symbol        2112 non-null   object
 1   Company Name  2113 non-null   object
dtypes: object(2)
memory usage: 33.1+ KB


In [36]:
data.head()

,Symbol,Company Name
0,RELIANCE,reliance industries limited
1,TCS,tata consultancy services limited
2,HDFCBANK,hdfc bank limited
3,INFY,infosys limited
4,ICICIBANK,icici bank limited


In [8]:
def filter_news(news_data, company_filter):
    news_data['Text'] = news_data['Text'].apply(str.lower)
    company_filter['Symbol'] = company_filter['Symbol'].apply(str).apply(str.lower)
    company_filter['Company Name'] = company_filter['Company Name'].apply(str).apply(str.lower)
    final_data = pd.DataFrame()
    news_data['Text'] = news_data['Text'].str.replace('[^\w\s]', ' ')
    news_data['Text'] = " "+news_data['Text']+" "

    # iterate all company names to find out the match
    for i in range(len(company_filter)-1):
        try:
            keyword = " "+company_filter["Symbol"][i]+" "
            CompanyName = " "+company_filter["Company Name"][i]+" "
            l=[" global "," wealth "," take "," focus "," worth "]
            if(keyword == 'ttl'):
                continue
            
            if(keyword in l):
                print(keyword)
                contain_values = news_data[(news_data['Text'].str.contains(keyword))&(news_data['Text'].str.contains(CompanyName))]
                contain_values['Symbol'] = company_filter["Symbol"][i]
                contain_values['Company Name'] = company_filter["Company Name"][i]
                final_data = final_data.append(contain_values, ignore_index=True)

            else:
                contain_values = news_data[(news_data['Text'].str.contains(keyword))|(news_data['Text'].str.contains(CompanyName))]
                contain_values['Symbol'] = company_filter["Symbol"][i]
                contain_values['Company Name'] = company_filter["Company Name"][i]
                final_data = final_data.append(contain_values, ignore_index=True)

        except:
            print("Company name is small at index ", l)

    return final_data
data1['Headline']=data1['Text']

In [22]:
df.head()

,Text,Date,Sentiment
0,Sebi to control unsolicited fin market advise ...,2023-02-24,1
1,"Sebi pitches change in rules for REITs, InvITs...",2023-02-24,1
2,Sri Lanka\s body approves renewable energy pro...,2023-02-24,1
3,Sebi likely to scrap small town-linked incenti...,2023-02-24,1
4,Indices post biggest weekly decline since June...,2023-02-24,-1


In [9]:
punct = string.punctuation
stopwords = list(STOP_WORDS)
def text_data_cleaning(sentence):
    sent = preprocess_text(sentence)
    doc = nlp(sent)
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return append_message(cleaned_tokens)
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''
    sentence = sen.lower()

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence) 
    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  
    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence


def append_message(text):
  str = " "
  return (str.join(text))


df['Tok_text'] = df['Text'].apply(preprocess_text)

tf_idf_vect = TfidfVectorizer()
X = df['Tok_text']
y = df['Sentiment']
# X = tf_idf_vect.fit_transform(X)

##splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train = tf_idf_vect.fit_transform(X_train)
X_test = tf_idf_vect.transform(X_test)
print(X_train.shape, X_test.shape)
## test data preprocessing
test_data['Tok_text'] = test_data['Text'].apply(preprocess_text)
test = test_data['Tok_text']
x_test = tf_idf_vect.transform(test_data['Tok_text'])
print(x_test.shape)

(21143, 9479) (5286, 9479)
(1435, 9479)


In [10]:
score = [0]

classifier = SGDClassifier()
param_grid = {
        'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2], # learning rate # number of epochs
        'loss': ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber'], # logistic regression,
        'penalty': ['l2'],
        'n_jobs': [-1]
    }

grid1 = GridSearchCV(classifier, param_grid, refit = True, verbose = 0)
    # fitting the model for grid search
grid1.fit(X_train, y_train)
print("For SDG classification")
score.append(grid1.best_score_)
print('Best Score: %s' % grid1.best_score_)
    # print best parameter after tuning
print(grid1.best_params_)
    # print how our model looks after hyper-parameter tuning
print(grid1.best_estimator_)

y_pred = grid1.predict(X_test)
# print("For SDG classification")
print(classification_report(y_test, y_pred))

classifier = LogisticRegression()
param_grid = {
        'C': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 1e0],
        'penalty': ['l2'],
        'n_jobs': [-1],
        'multi_class': ['multinomial'],
        'solver': ['lbfgs']
    }
grid2 = GridSearchCV(classifier, param_grid, refit = True, verbose = 0)
    # fitting the model for grid search
grid2.fit(X_train, y_train)
score.append(grid2.best_score_)
print("For Logistic Regression classification")
print('Best Score: %s' % grid2.best_score_)
    # print best parameter after tuning
print(grid2.best_params_)
    # print how our model looks after hyper-parameter tuning
print(grid2.best_estimator_)

y_pred = grid2.predict(X_test)
# print("For Logistic Regression classification")
print(classification_report(y_test, y_pred))


num = score.index(max(score))
if num == 1:
    y_pred = grid1.predict(x_test)
    check = pd.DataFrame({'Text':test_data['Text'],'Date':test_data['Date'],'Sentiment':y_pred})
    check.to_csv('check.csv')
elif num == 2:
    y_pred = grid2.predict(x_test)
    check = pd.DataFrame({'Text':test_data['Text'],'Date':test_data['Date'],'Sentiment':y_pred})
    check.to_csv('check.csv')
#     # y_pred = grid4.predict(x_test)


For SDG classification
Best Score: 0.913399281842967
{'alpha': 0.0001, 'loss': 'modified_huber', 'n_jobs': -1, 'penalty': 'l2'}
SGDClassifier(loss='modified_huber', n_jobs=-1)
              precision    recall  f1-score   support

          -1       0.90      0.84      0.87       812
           0       0.89      0.87      0.88      1191
           1       0.94      0.96      0.95      3283

    accuracy                           0.92      5286
   macro avg       0.91      0.89      0.90      5286
weighted avg       0.92      0.92      0.92      5286

For Logistic Regression classification
Best Score: 0.9167573628321634
{'C': 10, 'multi_class': 'multinomial', 'n_jobs': -1, 'penalty': 'l2', 'solver': 'lbfgs'}
LogisticRegression(C=10, multi_class='multinomial', n_jobs=-1)
              precision    recall  f1-score   support

          -1       0.91      0.87      0.89       812
           0       0.90      0.89      0.89      1191
           1       0.95      0.96      0.96      3283

  

In [6]:
df1 = pd.read_csv('check.csv')
df1.drop(columns=['Unnamed: 0'], axis=1,  inplace=True)
df1.rename(columns={'0' : 'Sentiment'},inplace=True)
index_name = df1[df1['Sentiment'].isna()==True].index
df1.drop(index_name,inplace=True)
print(df1.info())
print(df.info())
df.drop(columns=['Tok_text'], axis=1,  inplace=True)
fg = df.append(df1,ignore_index=True)
fg.info()
fg.to_csv('stock_data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       5000 non-null   object
 1   Date       5000 non-null   object
 2   Sentiment  5000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 117.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21429 entries, 0 to 21428
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       21429 non-null  object        
 1   Date       21429 non-null  datetime64[ns]
 2   Sentiment  21429 non-null  int64         
 3   Tok_text   21429 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 669.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26429 entries, 0 to 26428
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     -------------

C:\Users\kings\AppData\Local\Temp\ipykernel_12988\1174786404.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fg = df.append(df1,ignore_index=True)


In [32]:
# test_data = pd.read_csv('deb1.csv')
# test_data['Date'] = pd.to_datetime(test_data['Date'])
# test_data= test_data.iloc[:,1:4]
# test_data.rename(columns = {'Headline':'Text','Target':'Sentiment'}, inplace = True)
# test_data['Text'] = test_data['Text'].astype(str)
# test_data =test_data.replace(r'^\s*$', np.nan, regex=True)
# test_data = test_data[['Text','Date']].merge(df, on=['Text','Date'], how='left')
# test_data = test_data[test_data['Text'].str.contains("\?")==False]
# test_data = test_data.loc[test_data['Sentiment'].isnull() == True][:100]
# test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       100 non-null    object        
 1   Date       100 non-null    datetime64[ns]
 2   Sentiment  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 3.1+ KB


In [39]:
# test_data = pd.read_csv('soh1.csv')
# test_data['Date'] = pd.to_datetime(test_data['Date'])
# test_data= test_data.iloc[:,1:4]
# test_data.rename(columns = {'Headline':'Text','Target':'Sentiment'}, inplace = True)
# test_data['Text'] = test_data['Text'].astype(str)
# test_data =test_data.replace(r'^\s*$', np.nan, regex=True)
# test_data = test_data[['Text','Date']].merge(df, on=['Text','Date'], how='left')
# test_data = test_data[test_data['Text'].str.contains("\?")==False]
# test_data = test_data.loc[test_data['Sentiment'].isnull() == True]
# test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       4000 non-null   object        
 1   Date       4000 non-null   datetime64[ns]
 2   Sentiment  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 125.0+ KB


In [34]:
# test_data = pd.read_csv('say1.csv')
# test_data['Date'] = pd.to_datetime(test_data['Date'])
# test_data= test_data.iloc[:,1:4]
# test_data.rename(columns = {'Headline':'Text','Target':'Sentiment'}, inplace = True)
# test_data['Text'] = test_data['Text'].astype(str)
# test_data =test_data.replace(r'^\s*$', np.nan, regex=True)
# test_data = test_data[['Text','Date']].merge(df, on=['Text','Date'], how='left')
# test_data = test_data[test_data['Text'].str.contains("\?")==False]
# test_data = test_data.loc[test_data['Sentiment'].isnull() == True][:100]
# test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       100 non-null    object        
 1   Date       100 non-null    datetime64[ns]
 2   Sentiment  0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 3.1+ KB


In [70]:
y_pred = grid.predict(x_test)
check = pd.DataFrame({'Text':test_data['Text'],'Date':test_data['Date'],'Sentiment':y_pred})
check.to_csv('check.csv')

Using CNN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       100 non-null    object
 1   Date       100 non-null    object
 2   Sentiment  100 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 2.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2187 entries, 0 to 2186
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       2187 non-null   object        
 1   Date       2187 non-null   datetime64[ns]
 2   Sentiment  2187 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 51.4+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2387 entries, 0 to 2386
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       2387 non-null   object
 1   Date       2387 non-null   object
 2   Sentiment  2387 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 56.1+ KB


C:\Users\kings\AppData\Local\Temp\ipykernel_16836\1129578224.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fg = df.append(df1,ignore_index=True)


In [24]:
# df2 = test_data[['Text','Date']].merge(df1, on=['Text'], how='left')
# index_name = df2[df2['Sentiment'].isna()==True].index
# df2.drop(index_name,inplace=True)
# df2.head(30)

In [ ]:
# pos_msg = df[df['Sentiment'] == 1]
# zeo_msg = df[df['Sentiment']==0]
# nrg_msg = df[df['Sentiment']==-1]
# pos_msg_text = " ".join(pos_msg.Tok_text.to_numpy().tolist())
# zeo_msg_text = " ".join(zeo_msg.Tok_text.to_numpy().tolist())
# nrg_msg_text = " ".join(nrg_msg.Tok_text.to_numpy().tolist())

# pos_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(pos_msg_text)
# plt.figure(figsize=(16,10))
# plt.imshow(pos_msg_cloud, interpolation='bilinear')
# plt.axis('off') # turn off axis
# plt.show()
# zeo_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(zeo_msg_text)
# plt.figure(figsize=(16,10))
# plt.imshow(zeo_msg_cloud, interpolation='bilinear')
# plt.axis('off') # turn off axis
# plt.show()
# nrg_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(nrg_msg_text)
# plt.figure(figsize=(16,10))
# plt.imshow(nrg_msg_cloud, interpolation='bilinear')
# plt.axis('off') # turn off axis
# plt.show()


In [ ]:
# fix random seed for reproducibility
jj
tf.random.set_seed(7)

# load the dataset but only keep the top n words, zero the rest

top_words = 5000
#(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

X = []
y = df['Sentiment']
sentences = list(df['Text'])
for sen in sentences:
    X.append(preprocess_text(sen))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train)
X_train = word_tokenizer.texts_to_sequences(X_train)
X_test = word_tokenizer.texts_to_sequences(X_test)


vocab_length = len(word_tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


embeddings_dictionary = dict()
glove_file = open('/content/drive/My Drive/Colab Notebooks/a2_glove.6B.100d.txt', encoding="utf8")#/content/drive/MyDrive/Colab Notebooks/a2_glove.6B.100d.txt

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()


embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

print(embedding_matrix.shape)

In [ ]:
# Neural Network architecture

cnn_model = Sequential()

embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
cnn_model.add(embedding_layer)

cnn_model.add(Conv1D(128, 5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Model compiling

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(cnn_model.summary())

In [ ]:
cnn_model_history = cnn_model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

In [ ]:
score = cnn_model.evaluate(X_test, y_test, verbose=1)

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32]),
      activation='relu'))
  model.add(keras.layers.Dense(1, activation='relu'))
  model.compile(loss='mse')
  return model

In [ ]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
best_model = tuner.get_best_models()[0]

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0002, 
    epsilon=1e-08, 
    clipnorm=1.0),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )

In [ ]:
model.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))